In [240]:
import pandas as pd
import numpy as np
from persiantools import characters, digits
import jdatetime
import re
from glob import glob
import os

In [241]:
df_nemad = pd.read_excel(r'D:\Scrape\Cleaning\Script\Nemad_info.xlsx')

In [219]:
df_nemad[df_nemad.fa == 'بکاب']

,fa,en,company_name_English,company_name,stock_name,shenaseh_meli,firm_kind
74,بکاب,bekab,Yazd Jooshkab,صنايع‌جوشكاب‌يزد,بکاب,10100967098,NaN


In [242]:
df_nemad.stock_name = df_nemad.stock_name.apply(lambda x : characters.ar_to_fa(x))
df_nemad.fa = df_nemad.fa.apply(lambda x : characters.ar_to_fa(x))

In [243]:
def file_path(path):
    if __name__ == "__main__":
        cashflow = []

        for (root, dirs, files) in os.walk(path , topdown = True):
            if len(files) == 0:
                continue
            else:
                cashflow.append(root + '\\' + files[0])
        return cashflow

In [244]:
cashflow = file_path(r'D:\Scrape\Files\CashFlow')

In [245]:
len(cashflow)

2294

In [7]:
# def shape(kk):
#     l=pd.read_excel(kk)
#     a = l.shape
    
#     return a

# pd.Series(cashflow).apply(shape).unique()

In [8]:
# df1 = pd.read_excel(cashflow[0])
# df1 = df1[[0]]

# for i in cashflow:
#     df = pd.read_excel(i)
#     if df.shape != (0, 7):
#         df = df[[0 , 'title' , 'nemad']]
#         df1 = pd.concat([df1 , df])
# df1 = df1.drop_duplicates()
# # # df1.dropna().to_excel('CashFlow_UniqueValues.xlsx' , index=False)

In [9]:
# df1[df1[0] == 'بازپرداخت استقراض']

In [10]:
# df1[df1[0] == 'مالیات بر درآمد پرداختی']

In [246]:
def clean_drop_1(w):
    w.drop(w.columns[0], axis=1, inplace= True)
    w.dropna(axis=1 , thresh = 20 , inplace=True)
    # w.drop(w.columns[3], axis=1, inplace= True)

    # w.drop([1,3] , axis=0 , inplace=True)
    
    data_report = re.search(r'(\d+/\d+/\d+)', w.loc[0 , 'title']).group(1)
    w.rename(columns={0:"description" , 
                    # 1:f"reported_period {str(digits.fa_to_en(w.iloc[0,1].split(' ')[-1]))}" ,
                    1:f"reported_period {str(digits.fa_to_en(data_report))}" ,
                    w.columns[2]:f"variable_period {str(digits.fa_to_en(w.iloc[0,2].split(' ')[-1]))}"
                    } , inplace=True)

    return w

In [247]:
def new_col_1(i):

    i.insert(loc=2 , column='date_1' , value= re.sub("\/" , "-" , digits.fa_to_en(list(i.columns)[1].split(' ')[-1])))
    i.insert(loc=4 , column='date_2' , value= re.sub("\/" , "-" , digits.fa_to_en(list(i.columns)[3].split(' ')[-1])))

    i.drop([0] , axis=0 , inplace=True)
    i.reset_index(drop=True , inplace=True)

    # i.drop(columns=['percent_change'] , inplace=True)
    
    
    
    # nan_index= i[i['description'].isna()].index
    # nan_index= nan_index.to_list()
    # i.loc[nan_index , 'description']=['total_current_assets_befor' , 'total_current_liabilities_befor']

    
    
    i = i.replace(np.nan , '0')

    if 'حسابرسی شده' in i.iloc[0,0]:
        i.insert(loc=5 , column='audit_status_2' , value= 'audited')
    elif 'حسابرسی شده' in i.iloc[0,1]:
        i.insert(loc=5 , column='audit_status_2' , value= 'audited')
    else:
        i.insert(loc=5 , column='audit_status_2' , value= 'not_audited')

    if i.loc[0,'title'].find('حسابرسی شده') != -1:
        i.insert(loc=3 , column='audit_status_1' , value= "audited")
    else:
        i.insert(loc=3 , column='audit_status_1' , value= "not_audited")

    i.drop([0 , 1 , 2] , axis=0 , inplace=True)
    i.reset_index(drop=True , inplace=True)

    return i

In [248]:
def label_trans_1(y):

    y['description'].replace({'جریان­‌های نقدی حاصل از فعالیت‌های عملیاتی:':'cash_flows_from_operational_activities',
                                'نقد حاصل از عملیات':'cash_from_operations',
                                'پرداخت‌های نقدی بابت مالیات بر درآمد':'cash_payment_for_tax_on_income',
                                'جریان ‌خالص ‌ورود‌ (خروج) ‌نقد حاصل از فعالیت‌های ‌عملیاتی':'net_input_output_cash_flows_from_operational_activities',
                                'جریان‌­های نقدی حاصل از فعالیت‌های سرمایه‌گذاری:':'cash_flows_from_investment_activities',
                                'دریافت‌های نقدی حاصل از فروش دارایی‌های ثابت مشهود':'cash_receipts_from_selling_tangible_fixed_assets',
                                'پرداخت‌های نقدی برای خرید دارایی‌های ثابت مشهود':'cash_payment_for_purchase_tangible_fixed_assets',
                                'دریافت‌های نقدی حاصل از فروش دارایی‌های نگهداری‌شده برای فروش':'cash_receipts_from_selling_assets_hold_for_sale',
                                'دریافت‌های نقدی حاصل از فروش دارایی‌های نامشهود':'cash_receipts_from_selling_intangible_assets',
                                'پرداخت‌های نقدی برای خرید دارایی‌های نامشهود':'cash_payment_for_purchase_intangible_assets',
                                'دریافت‌های نقدی حاصل از فروش سرمايه‌گذاری‌های بلندمدت':'cash_receipts_from_selling_long_run_investment',
                                'پرداخت‌های نقدی برای تحصیل سرمايه‌گذاری‌های بلندمدت':'cash_payment_for_purchase_long_run_investment',
                                'دریافت‌های نقدی حاصل از فروش سرمایه‌گذاری در املاک':'cash_receipts_from_selling_investment_in_real_estate',
                                'پرداخت‌های نقدی برای تحصیل سرمایه‌گذاری در املاک':'cash_payment_for_purchase_investment_in_real_estate',
                                'دریافت‌های نقدی حاصل از فروش سرمايه‌گذاری‌های کوتاه‌مدت':'cash_receipts_from_selling_short_run_investment',
                                'پرداخت‌های نقدی برای تحصیل سرمایه‌گذاری‌های کوتاه‌مدت':'cash_payment_for_purchase_short_run_investment',
                                'پرداخت های نقدی بابت تسهیلات اعطایی به دیگران':'cash_payment_for_given_loan_to_others',
                                'دریافت‌های نقدی حاصل از استرداد تسهیلات اعطایی به دیگران':'cash_receipts_from_refund_given_loan_to_others',
                                'دریافت‌های نقدی حاصل از سود تسهیلات اعطایی به دیگران':'cash_receipts_from_given_loan_to_others_interest',
                                'دریافت‌های نقدی حاصل از سود‌ سهام':'cash_receipts_from_ stock_profit',
                                'دریافت‌های نقدی حاصل از سود سایر سرمایه‌گذاری‌ها':'cash_receipts_from_other_investment_profit',
                                'جريان خالص ورود (خروج) نقد حاصل از فعاليت‌های سرمایه‌گذاری':'net_input_output_cash_flows_from_investment_activities',
                                'جريان خالص ورود (خروج) نقد قبل از فعالیت‌های تامین مالی':'net_input_output_cash_flows_before_financial_activities',
                                'جریان­‌های نقدی حاصل از فعالیت‌های تامین مالی:':'cash_flows_from_financial_activities',
                                'دریافت‌های نقدی حاصل از افزايش سرمايه':'cash_receipts_from_capital_Increase',
                                'دریافت‌های نقدی حاصل از صرف سهام':'cash_receipts_from_ stock_premium',
                                'دریافت‌های نقدی حاصل از فروش سهام خزانه':'cash_receipts_from_selling_ treasury_stock',
                                'پرداخت‌های نقدی برای خرید سهام خزانه':'cash_payment_for_purchase_ treasury_stock',
                                'دریافت‌های نقدی حاصل از تسهيلات':'cash_receipts_from_loan',
                                'پرداخت‌های نقدی بابت اصل تسهيلات':'cash_payment_for_principle_of_loan',
                                'پرداخت‌های نقدی بابت سود تسهيلات':'cash_payment_for_interest_of_loan',
                                'دریافت‌های نقدی حاصل از انتشار اوراق مشارکت':'cash_receipts_from_Issuance_of_mosharekeh_securities',
                                'پرداخت‌های نقدی بابت اصل اوراق مشارکت':'cash_payment_for_principle_of_mosharekeh_securities',
                                'پرداخت‌های نقدی بابت سود اوراق مشارکت':'cash_payment_for_interest_of_mosharekeh_securities',
                                'دریافت‌های نقدی حاصل از انتشار اوراق خرید دین':'cash_receipts_from_Issuance_of_deyn_purchase_securities',
                                'پرداخت‌های نقدی بابت اصل اوراق خرید دین':'cash_payment_for_principle_of_deyn_purchase_securities',
                                'پرداخت‌های نقدی بابت سود اوراق خرید دین':'cash_payment_for_interest_of_deyn_purchase_securities',
                                'پرداخت‌های نقدی بابت اصل اقساط اجاره سرمایه‌ای':'cash_payment_for_principle_of_capital_lease_installments',
                                'پرداخت‌های نقدی بابت سود اجاره سرمایه‌ای':'cash_payment_for_interest_of_capital_lease_installments',
                                'پرداخت‌های نقدی بابت سود سهام':'cash_payment_for_ stock_profit',
                                'جريان خالص ورود (خروج) نقد حاصل از فعالیت‌های تامين مالی':'net_input_output_cash_flows_from_financial_activities',
                                'خالص افزايش (کاهش) در موجودی نقد':'net_increase_decrease_in_cash_balance',
                                'مانده موجودی نقد در ابتدای سال':'cash_balance_start_of_session',
                                'تاثير تغييرات نرخ ارز':'effect_of_foreign_exchange_change',
                                'مانده موجودی نقد در پايان سال':'cash_balance_end_of_session',
                                'معاملات غیرنقدی':'non_cash_transaction',
                                'فعالیت‌های عملیاتی':'operational_activities',
                                'جریان خالص ورود (خروج) وجه نقد ناشی از فعالیت‌های عملیاتی- عادی':'net_input_output_cash_flows_from_operational_activities_normal',
                                'جریان خالص ورود (خروج) وجه نقد ناشی از فعالیت‌های عملیاتی- استثنایی':'net_input_output_cash_flows_from_operational_activities_special',
                                'جریان خالص ورود (خروج) وجه نقد ناشی از فعالیت‌های عملیاتی':'net_input_output_cash_flows_from_operational_activities',
                                'بازده سرمایه گذاری‌ها و سود پرداختی بابت تأمین مالی':'net_input_output_cash_flows_from_investment_interest_and_payment_interest_for_finance',
                                'سود سهام دریافتی':'cash_receipts_from_ stock_profit',
                                'سود پرداختی بابت استقراض':'cash_payment_for_interest_of_borrowing',
                                'سود دریافتی بابت سایر سرمایه‌گذاری‌ها':'cash_receipts_from_other_investment_profit',
                                'سود سهام پرداختی':'cash_payment_for_ stock_profit',
                                'جریان خالص ورود (خروج) وجه نقد ناشی از بازده سرمایه‌گذاری‌ها و سود پرداختی بابت تأمین مالی':'net_input_output_cash_flows_from_investment_interest_and_payment_interest_for_finance',
                                'مالیات بر درآمد':'income_tax',
                                'مالیات بر درآمد پرداختی':'cash_payment_for_tax_on_income',
                                'فعالیت‌های سرمایه گذاری':'investment_activities',
                                'وجوه دریافتی بابت فروش دارایی‌های ثابت مشهود':'cash_receipts_from_selling_tangible_fixed_assets',
                                'وجوه پرداختی بابت خرید دارایی‌های ثابت مشهود':'cash_payment_for_purchase_tangible_fixed_assets',
                                'وجوه دریافتی بابت فروش دارایی‌های نامشهود':'cash_receipts_from_selling_intangible_assets',
                                'وجوه پرداختی بابت خرید دارایی‌های نامشهود‌':'cash_payment_for_purchase_intangible_assets',
                                'وجوه دریافتی بابت فروش سرمایه‌گذاری‌های بلندمدت':'cash_receipts_from_selling_long_run_investment',
                                'تسهیلات اعطایی به اشخاص':'cash_payment_for_given_loan_to_others',
                                'استرداد تسهیلات اعطایی به اشخاص':'cash_receipts_from_refund_given_loan_to_others',
                                'وجوه پرداختی بابت خرید سرمایه‌گذاری‌های بلندمدت':'cash_payment_for_purchase_long_run_investment',
                                'وجوه دریافتی بابت فروش سرمایه‌گذاری‌های کوتاه‌مدت':'cash_receipts_from_selling_short_run_investment',
                                'وجوه پرداختی بابت خرید سرمایه‌گذاری‌های کوتاه‌مدت':'cash_payment_for_purchase_short_run_investment',
                                'وجوه دریافتی بابت فروش سرمایه‌گذاری‌ در املاک':'cash_receipts_from_selling_investment_in_real_estate',
                                'وجوه پرداختی بابت خرید سرمایه‌گذاری در املاک':'cash_payment_for_purchase_investment_in_real_estate',
                                'جریان خالص ورود (خروج) وجه نقد ناشی از فعالیت‌های سرمایه‌گذاری':'net_input_output_cash_flows_from_investment_activities',
                                'جریان خالص ورود (خروج) وجه نقد قبل از فعالیت‌های تأمین مالی':'net_input_output_cash_flows_before_financial_activities',
                                'فعالیت‌های تأمین مالی':'financial_activities',
                                'وجوه دریافتی حاصل از افزایش سرمایه':'cash_receipts_from_capital_Increase',
                                'وجوه دریافتی بابت فروش سهام خزانه':'cash_receipts_from_selling_ treasury_stock',
                                'وجوه پرداختی بابت خرید سهام خزانه':'cash_payment_for_purchase_ treasury_stock',
                                'وجوه دریافتی حاصل از استقراض':'cash_receipts_from_borrowing',
                                'بازپرداخت اصل استقراض':'cash_payment_for_principle_of_borrowing',
                                'جریان خالص ورود (خروج) وجه نقد ناشی از فعالیت‌های تأمین مالی':'net_input_output_cash_flows_from_financial_activities',
                                'خالص افزایش (کاهش) در موجودی نقد':'net_increase_decrease_in_cash_balance',
                                'موجودی نقد در ابتدای دوره':'cash_balance_start_of_session',
                                'تآثیر تغییرات نرخ ارز':'effect_of_foreign_exchange_change',
                                'موجودی نقد در پایان دوره':'cash_balance_end_of_session',
                                'مبادلات غیرنقدی':'non_cash_transaction',
                                'بازده سرمایه‌گذاری‌ها و سود پرداختی بابت تأمین مالی':'net_input_output_cash_flows_from_investment_interest_and_payment_interest_for_finance',
                                'وجوه دریافتی بابت فروش دارایی‌های نگهداری شده برای فروش':'cash_receipts_from_selling_assets_hold_for_sale',
                                'وجوه دریافتی حاصل از فروش سهام خزانه':'cash_receipts_from_selling_ treasury_stock',
                                'جریان خالص ورود(خروج) وجه نقد ناشی از فعالیت‌های عملیاتی-عادی':'net_input_output_cash_flows_from_operational_activities_normal',
                                'جریان خالص ورود(خروج) وجه نقد ناشی از فعالیت‌های عملیاتی - استثنایی':'net_input_output_cash_flows_from_operational_activities_special',
                                'جریان خالص ورود(خروج) وجه نقد ناشی از فعالیت‌های عملیاتی':'net_input_output_cash_flows_from_operational_activities',
                                'بازده سرمایه گذاری‌ها و سود پرداختی بابت تامین مالی':'net_input_output_cash_flows_from_investment_interest_and_payment_interest_for_finance',
                                'سود دریافتی بابت سایر سرمایه گذاری‌ها':'cash_receipts_from_other_investment_profit',
                                'جریان خالص ورود(خروج) وجه نقد ناشی از بازده سرمایه گذاری‌ها و سود پرداختی بابت تامین مالی':'net_input_output_cash_flows_from_investment_interest_and_payment_interest_for_finance',
                                'وجوه پرداختی بابت تحصیل دارایی‌های ثابت مشهود':'cash_payment_for_purchase_tangible_fixed_assets',
                                'وجوه پرداختی بابت تحصیل دارایی‌های نامشهود':'cash_payment_for_purchase_intangible_assets',
                                'وجوه دریافتی بابت فروش سرمایه گذاری‌های بلند مدت':'cash_receipts_from_selling_long_run_investment',
                                'وجوه پرداختی بابت تحصیل سرمایه گذاری‌های بلند مدت':'cash_payment_for_purchase_long_run_investment',
                                'وجوه دریافتی بابت فروش سرمایه گذاری‌های کوتاه مدت':'cash_receipts_from_selling_short_run_investment',
                                'وجوه پرداختی بابت تحصیل سرمایه گذاری‌های کوتاه مدت':'cash_payment_for_purchase_short_run_investment',
                                'جریان خالص ورود(خروج) وجه نقد ناشی از فعالیت‌های سرمایه گذاری':'net_input_output_cash_flows_from_investment_activities',
                                'جریان خالص ورود(خروج) وجه نقد قبل از فعالیت‌های تامین مالی':'net_input_output_cash_flows_before_financial_activities',
                                'فعالیت‌های تامین مالی':'financial_activities',
                                'بازپرداخت استقراض':'cash_payment_for_principle_of_borrowing',
                                'جریان خالص ورود(خروج) وجه نقد ناشی از فعالیت‌های تامین مالی':'net_input_output_cash_flows_from_financial_activities',
                                'خالص افزایش(کاهش) در وجه نقد':'net_increase_decrease_in_cash_balance',
                                'مانده وجه نقد در ابتدای دوره':'cash_balance_start_of_session',
                                'تاثیر تغییرات نرخ ارز':'effect_of_foreign_exchange_change',
                                'وجه نقد در پایان دوره':'cash_balance_end_of_session',
                                'جریان های نقدی حاصل از فعاليت‌هاي عملياتي:':'cash_flows_from_operational_activities',
                                'جريان ‌خالص ‌ورود‌ (خروج) ‌نقد حاصل از فعاليت‌هاي ‌عملياتي':'net_input_output_cash_flows_from_operational_activities',
                                'جریان­های نقدی حاصل از فعاليت‌هاي سرمايه‌گذاري:':'cash_flows_from_investment_operations',
                                'دریافت‌های نقدی حاصل از فروش دارايي‌هاي زیستی مولد':'cash_receipts_from_selling_productive_bio_assets',
                                'پرداخت‌های نقدی برای خرید دارايي‌هاي زیستی مولد':'cash_payment_for_purchase_productive_bio_assets',
                                'دریافت‌های نقدی حاصل از فروش دارايي‌هاي ثابت مشهود':'cash_receipts_from_selling_tangible_fixed_assets',
                                'پرداخت‌های نقدی برای خرید دارايي‌هاي ثابت مشهود':'cash_payment_for_purchase_tangible_fixed_assets',
                                'دریافت‌های نقدی حاصل از فروش دارايي‌هاي نگهداری‌شده برای فروش':'cash_receipts_from_selling_assets_hold_for_sale',
                                'دریافت‌های نقدی حاصل از فروش دارايي‌هاي نامشهود':'cash_receipts_from_selling_intangible_assets',
                                'پرداخت‌های نقدی برای خرید دارايي‌هاي نامشهود':'cash_payment_for_purchase_intangible_assets',
                                'دریافت‌های نقدی حاصل از فروش سرمايه‌گذاري‌هاي بلندمدت':'cash_receipts_from_selling_long_run_investment',
                                'پرداخت‌های نقدی برای تحصیل سرمايه‌گذاري‌هاي بلندمدت':'cash_payment_for_purchase_long_run_investment',
                                'دریافت‌های نقدی حاصل از فروش سرمايه‌گذاري در املاک':'cash_receipts_from_selling_investment_in_real_estate',
                                'پرداخت‌های نقدی برای تحصیل سرمايه‌گذاري‌ در املاک':'cash_payment_for_purchase_investment_in_real_estate',
                                'دریافت‌های نقدی حاصل از فروش سرمايه‌گذاري‌هاي کوتاه‌مدت':'cash_receipts_from_selling_short_run_investment',
                                'پرداخت‌های نقدی برای تحصیل سرمايه‌گذاري‌هاي کوتاه‌مدت':'cash_payment_for_purchase_short_run_investment',
                                'تسهیلات اعطایی به دیگران':'cash_payment_for_given_loan_to_others',
                                'استرداد تسهیلات اعطایی به دیگران':'cash_receipts_from_refund_given_loan_to_others',
                                'سود دریافتی بابت تسهیلات اعطایی به دیگران':'cash_receipts_from_given_loan_to_others_interest',
                                'جريان خالص ورود (خروج) نقد حاصل از فعاليت‌هاي سرمايه‌گذاري':'net_input_output_cash_flows_from_investment_activities',
                                'جريان خالص ورود (خروج) نقد قبل از فعاليت‌هاي تامين مالي':'net_input_output_cash_flows_before_financial_activities',
                                'جریان­های نقدی حاصل از فعاليت‌هاي تامين مالي:':'cash_flows_from_financial_activities',
                                'جريان خالص ورود (خروج) نقد حاصل از فعاليت‌هاي تامين مالي':'net_input_output_cash_flows_from_financial_activities',
                                'معاملات غيرنقدي':'non_cash_transaction'} , inplace=True)



    y = y.iloc[0:(y[y.description == 'non_cash_transaction'].index +1)[0] , :]
    return y


In [131]:
# x_concat_1(char_edit_1(time_edit_1(col_edit_1(label_trans_1(new_col_1(clean_drop_1(pd.read_excel(cashflow[292])))))))).head(50)
# cashflow[290]

total_value_1(x_concat_1(char_edit_1(time_edit_1(col_edit_1(label_trans_1(new_col_1(clean_drop_1(pd.read_excel(cashflow[292]))))))))).head(50)

,description,value,date,audit_status,is_total_value,national_code,nemad,company_name,reported_period,reported_date_j,reported_date_m,is_corrected,send_time,publish_time,table_title
0,operational_activities,0,1398-06-31,audited,0,10530196515,cefars,Fars Dev.,12_months_interim,1398-06-31,2019-09-22,0,1398/10/17 10:23:02,1398/10/17 10:23:02,cash_flows
1,net_input_output_cash_flows_from_operational_a...,"183,478",1398-06-31,audited,0,10530196515,cefars,Fars Dev.,12_months_interim,1398-06-31,2019-09-22,0,1398/10/17 10:23:02,1398/10/17 10:23:02,cash_flows
2,net_input_output_cash_flows_from_operational_a...,0,1398-06-31,audited,0,10530196515,cefars,Fars Dev.,12_months_interim,1398-06-31,2019-09-22,0,1398/10/17 10:23:02,1398/10/17 10:23:02,cash_flows
3,net_input_output_cash_flows_from_operational_a...,"183,478",1398-06-31,audited,1,10530196515,cefars,Fars Dev.,12_months_interim,1398-06-31,2019-09-22,0,1398/10/17 10:23:02,1398/10/17 10:23:02,cash_flows
4,net_input_output_cash_flows_from_investment_in...,0,1398-06-31,audited,0,10530196515,cefars,Fars Dev.,12_months_interim,1398-06-31,2019-09-22,0,1398/10/17 10:23:02,1398/10/17 10:23:02,cash_flows
5,cash_receipts_from_ stock_profit,"23,963",1398-06-31,audited,0,10530196515,cefars,Fars Dev.,12_months_interim,1398-06-31,2019-09-22,0,1398/10/17 10:23:02,1398/10/17 10:23:02,cash_flows
6,cash_payment_for_interest_of_borrowing,"(45,518)",1398-06-31,audited,0,10530196515,cefars,Fars Dev.,12_months_interim,1398-06-31,2019-09-22,0,1398/10/17 10:23:02,1398/10/17 10:23:02,cash_flows
7,cash_receipts_from_other_investment_profit,"1,926",1398-06-31,audited,0,10530196515,cefars,Fars Dev.,12_months_interim,1398-06-31,2019-09-22,0,1398/10/17 10:23:02,1398/10/17 10:23:02,cash_flows
8,cash_payment_for_ stock_profit,"(62,416)",1398-06-31,audited,0,10530196515,cefars,Fars Dev.,12_months_interim,1398-06-31,2019-09-22,0,1398/10/17 10:23:02,1398/10/17 10:23:02,cash_flows
9,net_input_output_cash_flows_from_investment_in...,"(82,045)",1398-06-31,audited,0,10530196515,cefars,Fars Dev.,12_months_interim,1398-06-31,2019-09-22,0,1398/10/17 10:23:02,1398/10/17 10:23:02,cash_flows


In [249]:
def col_edit_1(t):

    

    t.insert(loc= 10, column= "national_code", value= list(df_nemad.loc[df_nemad['fa']== t.loc[0 , 'nemad']]['shenaseh_meli'])[0])
    t.replace(t['company_name'].values, list(df_nemad.loc[df_nemad['fa']== t.loc[0 , 'nemad'] ]['company_name_English'])[0], inplace=True)
    t.replace(t['nemad'].values, list(df_nemad.loc[df_nemad['fa']== t.loc[0 , 'nemad']]['en'])[0] , inplace=True)
    

    t.replace(t['table_title'].values, "cash_flows" , inplace=True)

    if t.loc[0,'title'].find('دوره ۶ ماهه') != -1:
        t.insert(loc=13 , column = 'reported_period' , value = "6_months_interim")
    elif t.loc[0,'title'].find('دوره ۳ ماهه') != -1:
        t.insert(loc=13 , column='reported_period' , value= "3_months_interim")
    elif t.loc[0,'title'].find('دوره ۹ ماهه') != -1:
        t.insert(loc=13 , column='reported_period' , value= "9_months_interim")
    else:
        t.insert(loc=13 , column='reported_period' , value= "12_months_interim")

    return t

In [250]:
def to_g(x):
    x=jdatetime.datetime(int(x.split('-')[0]) , int(x.split('-')[1]),int(x.split('-')[2])).togregorian()
    return x

In [251]:
def time_edit_1(r):
    r.insert(loc=11 , column="reported_date_j" , value =re.sub("\/" , "-" , digits.fa_to_en(re.search(r'(\d+/\d+/\d+)', r.loc[0 , 'title']).group(1))) )
    r.insert(loc=12 , column="reported_date_m" , value= r['reported_date_j'].apply(lambda x:to_g(x)))
    return r

In [252]:
def char_edit_1(ch):

    if ch.loc[0 , 'title'].find('اصلاحیه') != -1:
        ch.insert(loc=17 , column= 'is_corrected' , value='1')
    else:
        ch.insert(loc=17 , column= 'is_corrected' , value='0')
        
    ch.drop(columns=['title'] , inplace=True)

    ch[ch.columns[4]]= ch[ch.columns[4]].apply(lambda x: digits.fa_to_en(x))
    ch[ch.columns[1]]= ch[ch.columns[1]].apply(lambda x: digits.fa_to_en(x))

    ch['publish_time']= ch['publish_time'].apply(lambda x: digits.fa_to_en(x))
    ch['send_time']= ch['send_time'].apply(lambda x: digits.fa_to_en(x))

    return ch

In [253]:
def x_concat_1(table):

    table_1= table.filter(['description' , table.columns[1] , 'date_1' , 'audit_status_1' , 'national_code' , 'nemad' , 'company_name' ,
                     'reported_period' , 'reported_date_j' , 'reported_date_m' , 'reported_audit_status' , 'is_corrected' , 'talfighi' , 'send_time' , 'publish_time' , 'table_title'])
    table_2= table.filter(['description' , table.columns[4] , 'date_2' , 'audit_status_2' , 'national_code' , 'nemad' , 'company_name' ,
                     'reported_period' , 'reported_date_j' , 'reported_date_m' , 'reported_audit_status' , 'is_corrected' , 'talfighi' ,'send_time' , 'publish_time' , 'table_title'])

    table_1.rename(columns={table.columns[1]:'value' , 'date_1':'date' , 'audit_status_1':'audit_status'} , inplace=True)
    table_2.rename(columns={table.columns[4]:'value' , 'date_2':'date' , 'audit_status_2':'audit_status'} , inplace=True)

    data_final= pd.concat([table_1 , table_2 ] , ignore_index=True)

    return data_final

In [254]:
def total_value_1(p):
    
    p.drop(p[p.description.str.contains("شرح")].index , inplace=True)
    p.drop(p[p.description.str.contains("/")].index , inplace=True)
    p.drop(p[p.value.str.contains("/")].index , inplace=True)

    p.insert(loc=4 , column='is_total_value' , value= '0')


    index1_all=p[p.description.str.contains('net')].index

    index1_all=index1_all.to_list()

    index2=p[p.description.str.contains('balance')].index
    index2=index2.to_list()

    p.loc[index2,'is_total_value']= 1
    p.loc[index1_all,'is_total_value']= 1


    p.loc[p[p.description.str.contains('net_input_output_cash_flows_from_operational_activities_normal')].index , 'is_total_value'] = '0'
    p.loc[p[p.description.str.contains('net_input_output_cash_flows_from_operational_activities_special')].index , 'is_total_value'] = '0'
    p.loc[p[p.description.str.contains('net_input_output_cash_flows_from_investment_interest_and_payment_interest_for_finance')].index , 'is_total_value'] = '0'

    # p.loc[p.description == 'net_input_output_cash_flows_from_operational_activities_normal' , 'is_total_value'] = '0'
    # p.loc[p.description == 'net_input_output_cash_flows_from_operational_activities_special' , 'is_total_value'] = '0'
    # p.loc[p.description == 'net_input_output_cash_flows_from_investment_interest_and_payment_interest_for_finance' , 'is_total_value'] = '0'


    p.drop(p[p.description == '0' ].index , inplace = True)


    return p


In [286]:
coding_df = pd.read_excel(r'D:\CompanyDB\MetaData\CashFlowCoding\CashFlowCodingForDB.xlsx')
coding_df = coding_df[['en' , 'Code']]
coding_df.columns = ['description' , 'description_code']

In [287]:
def merge(df):

    df1 = df.merge(coding_df , on='description' , how = 'left')
    df1 = df1.filter(['description' , 'value' , 'date' , 'description_code' , 'is_total_value', 'audit_status' , 'national_code' , 'nemad' , 'company_name' ,
                     'reported_period' , 'reported_date_j' , 'reported_date_m' , 'is_corrected'  ,'send_time' , 'publish_time' , 'table_title'])
    return df1

In [288]:
def save(final_df):

    final_df.reset_index(drop = True , inplace=True )

    if final_df.loc[0 , 'is_corrected']=='1':
        k='_correction_'
    else:
        k='_'
          
    directory = r"D:\Scrape\Cleaning\cleaned\CashFlowCoded"
    file_name = list(df_nemad.loc[df_nemad['en'] == final_df.loc[0 , 'nemad']]['en'])[0]
    path_join = os.path.join(directory, file_name)
    try:
        os.mkdir(path_join)
    except :
        print('Cant')
           
    
    final_df.to_excel(f"{path_join}/{final_df.iloc[0,6]}_{final_df.iloc[0,7]}_{final_df.iloc[0,10]}_{final_df.iloc[0,5]}{k}cashflow.xlsx" , index=False)



In [37]:
save(total_value_1(x_concat_1(char_edit_1(time_edit_1(col_edit_1(new_col_1(clean_drop_1(pd.read_excel(cashflow[300])))))))))

Cant


In [289]:
def export(path):
    save(merge(total_value_1(x_concat_1(char_edit_1(time_edit_1(col_edit_1(label_trans_1(new_col_1(clean_drop_1(pd.read_excel(path)))))))))))

In [290]:
errorlist_1 = []
def run_function(series):
    try:
        pd.Series(series).apply(export)
    except:
        errorlist_1.append(series)

In [291]:
errorlist_1 = []
pd.Series(cashflow).apply(run_function)

Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant


0       None
1       None
2       None
3       None
4       None
        ... 
2289    None
2290    None
2291    None
2292    None
2293    None
Length: 2294, dtype: object

In [292]:
# errorlist_1[20:50]
len(errorlist_1)

# errorlist_1[33]

672

In [235]:
# pd.read_excel(errorlist_1[25])


total_value_1(x_concat_1(char_edit_1(time_edit_1(col_edit_1(label_trans_1(new_col_1(clean_drop_1(pd.read_excel(errorlist_1[33]))))))))).head(50)

,description,value,date,audit_status,is_total_value,national_code,nemad,company_name,reported_period,reported_date_j,reported_date_m,is_corrected,send_time,publish_time,table_title
1,operational_activities,0,1398-06-31,not_audited,0,10100967098,bekab,Yazd Jooshkab,12_months_interim,1398-06-31,2019-09-22,0,1398/08/29 16:45:11,1398/08/29 16:45:11,cash_flows
2,net_input_output_cash_flows_from_operational_a...,"15,986",1398-06-31,not_audited,0,10100967098,bekab,Yazd Jooshkab,12_months_interim,1398-06-31,2019-09-22,0,1398/08/29 16:45:11,1398/08/29 16:45:11,cash_flows
3,net_input_output_cash_flows_from_operational_a...,0,1398-06-31,not_audited,0,10100967098,bekab,Yazd Jooshkab,12_months_interim,1398-06-31,2019-09-22,0,1398/08/29 16:45:11,1398/08/29 16:45:11,cash_flows
4,net_input_output_cash_flows_from_operational_a...,"15,986",1398-06-31,not_audited,1,10100967098,bekab,Yazd Jooshkab,12_months_interim,1398-06-31,2019-09-22,0,1398/08/29 16:45:11,1398/08/29 16:45:11,cash_flows
5,net_input_output_cash_flows_from_investment_in...,0,1398-06-31,not_audited,0,10100967098,bekab,Yazd Jooshkab,12_months_interim,1398-06-31,2019-09-22,0,1398/08/29 16:45:11,1398/08/29 16:45:11,cash_flows
6,cash_receipts_from_ stock_profit,0,1398-06-31,not_audited,0,10100967098,bekab,Yazd Jooshkab,12_months_interim,1398-06-31,2019-09-22,0,1398/08/29 16:45:11,1398/08/29 16:45:11,cash_flows
7,cash_payment_for_interest_of_borrowing,0,1398-06-31,not_audited,0,10100967098,bekab,Yazd Jooshkab,12_months_interim,1398-06-31,2019-09-22,0,1398/08/29 16:45:11,1398/08/29 16:45:11,cash_flows
8,cash_receipts_from_other_investment_profit,25,1398-06-31,not_audited,0,10100967098,bekab,Yazd Jooshkab,12_months_interim,1398-06-31,2019-09-22,0,1398/08/29 16:45:11,1398/08/29 16:45:11,cash_flows
9,cash_payment_for_ stock_profit,(517),1398-06-31,not_audited,0,10100967098,bekab,Yazd Jooshkab,12_months_interim,1398-06-31,2019-09-22,0,1398/08/29 16:45:11,1398/08/29 16:45:11,cash_flows
10,net_input_output_cash_flows_from_investment_in...,(492),1398-06-31,not_audited,0,10100967098,bekab,Yazd Jooshkab,12_months_interim,1398-06-31,2019-09-22,0,1398/08/29 16:45:11,1398/08/29 16:45:11,cash_flows


In [234]:
clean_drop_1(pd.read_excel(errorlist_1[33]))

,description,reported_period 1398/06/31,variable_period 1397/06/31,nemad,company_name,title,send_time,publish_time,table_title
0,شرح,واقعی دوره منتهی به ۱۳۹۸/۰۶/۳۱,واقعی سال مالی منتهی به ۱۳۹۷/۰۶/۳۱,بکاب,صنایع جوشکاب یزد,صورت‌های مالی سال مالی منتهی به ۱۳۹۸/۰۶/۳۱ (حس...,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,Cash Flow
1,NaN,NaN,NaN,بکاب,صنایع جوشکاب یزد,صورت‌های مالی سال مالی منتهی به ۱۳۹۸/۰۶/۳۱ (حس...,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,Cash Flow
2,NaN,NaN,NaN,بکاب,صنایع جوشکاب یزد,صورت‌های مالی سال مالی منتهی به ۱۳۹۸/۰۶/۳۱ (حس...,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,Cash Flow
3,حسابرسی نشده,NaN,NaN,بکاب,صنایع جوشکاب یزد,صورت‌های مالی سال مالی منتهی به ۱۳۹۸/۰۶/۳۱ (حس...,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,Cash Flow
4,NaN,NaN,NaN,بکاب,صنایع جوشکاب یزد,صورت‌های مالی سال مالی منتهی به ۱۳۹۸/۰۶/۳۱ (حس...,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,Cash Flow
5,فعالیت‌های عملیاتی,NaN,NaN,بکاب,صنایع جوشکاب یزد,صورت‌های مالی سال مالی منتهی به ۱۳۹۸/۰۶/۳۱ (حس...,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,Cash Flow
6,جریان خالص ورود (خروج) وجه نقد ناشی از فعالیت‌...,"۱۵,۹۸۶","۲,۳۰۳",بکاب,صنایع جوشکاب یزد,صورت‌های مالی سال مالی منتهی به ۱۳۹۸/۰۶/۳۱ (حس...,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,Cash Flow
7,جریان خالص ورود (خروج) وجه نقد ناشی از فعالیت‌...,۰,۰,بکاب,صنایع جوشکاب یزد,صورت‌های مالی سال مالی منتهی به ۱۳۹۸/۰۶/۳۱ (حس...,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,Cash Flow
8,جریان خالص ورود (خروج) وجه نقد ناشی از فعالیت‌...,"۱۵,۹۸۶","۲,۳۰۳",بکاب,صنایع جوشکاب یزد,صورت‌های مالی سال مالی منتهی به ۱۳۹۸/۰۶/۳۱ (حس...,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,Cash Flow
9,بازده سرمایه گذاری‌ها و سود پرداختی بابت تأمین...,NaN,NaN,بکاب,صنایع جوشکاب یزد,صورت‌های مالی سال مالی منتهی به ۱۳۹۸/۰۶/۳۱ (حس...,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,۱۳۹۸/۰۸/۲۹ ۱۶:۴۵:۱۱,Cash Flow
